In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score


In [3]:
df = pd.read_csv('train.csv',usecols=['Age','Fare','SibSp','Parch','Survived'])

In [4]:
df

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500
...,...,...,...,...,...
886,0,27.0,0,0,13.0000
887,1,19.0,0,0,30.0000
888,0,NaN,1,2,23.4500
889,1,26.0,0,0,30.0000


In [5]:
df.isnull().sum()

Survived      0
Age         177
SibSp         0
Parch         0
Fare          0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.sample(10)

,Survived,Age,SibSp,Parch,Fare
150,0,51.0,0,0,12.5250
852,0,9.0,1,1,15.2458
86,0,16.0,1,3,34.3750
664,1,20.0,1,0,7.9250
657,0,32.0,1,1,15.5000
348,1,3.0,1,1,15.9000
328,1,31.0,1,1,20.5250
880,1,25.0,0,1,26.0000
830,1,15.0,1,0,14.4542
887,1,19.0,0,0,30.0000


In [8]:
df['family']=df['SibSp']+df['Parch']

In [9]:
df.sample(10)

,Survived,Age,SibSp,Parch,Fare,family
280,0,65.0,0,0,7.7500,0
172,1,1.0,1,1,11.1333,2
130,0,33.0,0,0,7.8958,0
651,1,18.0,0,1,23.0000,1
120,0,21.0,2,0,73.5000,2
218,1,32.0,0,0,76.2917,0
341,1,24.0,3,2,263.0000,5
119,0,2.0,4,2,31.2750,6
294,0,24.0,0,0,7.8958,0
104,0,37.0,2,0,7.9250,2


In [10]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [11]:
df.sample(10)

,Survived,Age,Fare,family
102,0,21.0,77.2875,1
818,0,43.0,6.4500,0
378,0,20.0,4.0125,0
383,1,35.0,52.0000,1
83,0,28.0,47.1000,0
884,0,25.0,7.0500,0
94,0,59.0,7.2500,0
787,0,8.0,29.1250,5
372,0,19.0,8.0500,0
24,0,8.0,21.0750,4


In [12]:
X = df.drop(columns='Survived')
y = df['Survived']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
X_train

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0
...,...,...,...
92,46.0,61.1750,1
134,25.0,13.0000,0
337,41.0,134.5000,0
548,33.0,20.5250,2


In [15]:
# without binarization

clf = DecisionTreeClassifier()

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_test,y_pred)

0.6083916083916084

In [16]:
np.mean(cross_val_score(clf,X,y,cv=10))

0.6401408450704226

In [17]:
# Applying Binarization

from sklearn.preprocessing import Binarizer

In [19]:
trf = ColumnTransformer([
    ('bin', Binarizer(copy=False), ['family']),
],remainder='passthrough')

In [20]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [22]:
pd.DataFrame(X_train_trf,columns=['family','age','fare']).sample(10)

,family,age,fare
526,0.0,28.0,10.5000
135,0.0,36.0,13.0000
260,1.0,50.0,106.4250
378,0.0,30.0,13.0000
227,0.0,45.0,7.7500
361,0.0,30.0,8.6625
141,1.0,20.0,9.8250
235,1.0,44.0,26.0000
128,1.0,54.0,78.2667
58,0.0,27.0,6.9750


In [23]:
# with binarization

clf = DecisionTreeClassifier()

clf.fit(X_train_trf,y_train)

y_pred = clf.predict(X_test_trf)

accuracy_score(y_test,y_pred)

0.6153846153846154

In [24]:
X_trf = trf.fit_transform(X)

In [25]:
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

0.6318661971830986